In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import joblib
import xgboost as xgb
import pandas as pd
import pickle

In [18]:
df = joblib.load("df_filtered.pkl")

In [20]:
df.head()

,brand,model,model_year,milage,fuel_type,accident,clean_title,price,categorized_model,transmission_type,Luxury_Brand,cat_ext_col,ext_col_metal,cat_int_col,milage_year,engine_size_y
0,Ford,Utility Police Interceptor Base,2013,51000.0,E85 Flex Fuel,At least 1 accident or damage reported,1,10300.0,Luxury SUV,Auto,0,Black,0,Black,4250.000000,3.7
1,Hyundai,Palisade SEL,2021,34742.0,Gasoline,At least 1 accident or damage reported,1,38005.0,SUV,Manual,0,Blue,0,Silver_Gray,8685.500000,3.8
2,Lexus,RX 350 RX 350,2022,22372.0,Gasoline,None reported,0,54598.0,Luxury SUV,Manual,0,Blue,0,Black,7457.333333,3.5
3,INFINITI,Q50 Hybrid Sport,2015,88900.0,Hybrid,None reported,1,15500.0,Luxury SUV,Auto,0,Black,0,Black,8890.000000,3.5
4,Audi,Q3 45 S line Premium Plus,2021,9835.0,Gasoline,None reported,0,34999.0,Luxury SUV,Manual,0,White,1,Black,2458.750000,2.0


In [22]:
df["price"].describe()

count     3696.000000
mean     33947.139881
std      21311.687767
min       2000.000000
25%      17000.000000
50%      29995.000000
75%      45999.000000
max      99000.000000
Name: price, dtype: float64

In [24]:
X = df.drop(columns=['price'])
y = df['price']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [157]:
col_cat = ["brand","fuel_type","cat_ext_col","ext_col_metal",
                                "cat_int_col","accident","clean_title","transmission_type",
                                "categorized_model"]
Encoder = OneHotEncoder(drop="first",dtype=float)
Transformer = ColumnTransformer([("One Hot Encoder", Encoder,col_cat)], remainder="passthrough")

In [159]:
params = {'colsample_bylevel': 1,
 'colsample_bytree': 0.7,
 'gamma': 0.01,
 'learning_rate': 0.01,
 'max_depth': 4,
 'n_estimators': 1000,
 'n_jobs': -1,
 'objective': 'reg:squarederror',
 'random_state': 42,
 'reg_alpha': 0.2,
 'reg_lambda': 0.1}

In [161]:
xgb_model = xgb.XGBRegressor(**params)

In [163]:
Pipeline_xgb = make_pipeline(Transformer,xgb_model)

In [165]:
Pipeline_xgb.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('One Hot Encoder',
                                                  OneHotEncoder(drop='first',
                                                                dtype=<class 'float'>),
                                                  ['brand', 'fuel_type',
                                                   'cat_ext_col',
                                                   'ext_col_metal',
                                                   'cat_int_col', 'accident',
                                                   'clean_title',
                                                   'transmission_type',
                                                   'categorized_model'])])),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=Non...
                              feature_types=None, gamma=0.01, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.01,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=4, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=1000, n_jobs=-1,
                              num_parallel_tree=None, random_state=42, ...))])

In [185]:
filename = 'xgb_car_price_pipeline.pkl'
pickle.dump(Pipeline_xgb, open(filename, 'wb'))

In [205]:
X.columns

Index(['brand', 'model_year', 'milage', 'fuel_type', 'accident', 'clean_title',
       'categorized_model', 'transmission_type', 'Luxury_Brand', 'cat_ext_col',
       'ext_col_metal', 'cat_int_col', 'milage_year', 'engine_size_y'],
      dtype='object')